# HSV 기반 객체 탐지 테스트 노트북

이 노트북은 HSV 색상 기반 객체 탐지 프로그램의 주요 기능을 테스트합니다.
- 화면 캡처 기능
- HSV 색상 필터링
- 객체 검출 및 바운딩 박스 생성

## 0. 필요한 패키지 설치

프로젝트 실행에 필요한 패키지를 설치합니다.

In [21]:
# 필요한 패키지 확인 및 설치
import sys
import subprocess
import importlib

# 필요한 패키지 목록 - VS Code 위젯 지원을 위한 패키지 추가
required_packages = [
    'numpy', 
    'matplotlib', 
    'ipywidgets>=7.6.0', 
    'mss', 
    'numba', 
    'setuptools',
    'jupyterlab_widgets',  # Jupyter 위젯 지원
    'ipympl',              # matplotlib 상호작용
    'widgetsnbextension'   # 노트북 위젯 확장
]

# 이미 설치된 패키지와 설치가 필요한 패키지 확인
packages_to_install = []
installed_packages = []

for package in required_packages:
    try:
        # 패키지 이름에서 버전 부분 분리
        package_name = package.split('>=')[0].split('[')[0]
        # 패키지 가져오기 시도
        importlib.import_module(package_name)
        installed_packages.append(package)
    except ImportError:
        packages_to_install.append(package)

# 결과 메시지 초기화
if not packages_to_install:
    print("✅ 모든 필요한 패키지가 이미 설치되어 있습니다.")
else:
    print(f"🔄 설치가 필요한 패키지: {', '.join(packages_to_install)}")
    print(f"🔄 이미 설치된 패키지: {', '.join(installed_packages)}")
    
    # 필요한 패키지만 설치
    try:
        !pip install {' '.join(packages_to_install)}
        print(f"✅ 패키지 설치가 완료되었습니다: {', '.join(packages_to_install)}")
        
        # 주피터 위젯 확장 활성화
        !jupyter nbextension enable --py widgetsnbextension
        print("✅ 위젯 확장이 활성화되었습니다.")
    except Exception as e:
        print(f"❌ 패키지 설치 중 오류 발생: {e}")

# VS Code용 추가 설정 확인
print("\n🔍 VS Code 환경 확인 중...")
in_vscode = 'vscode' in sys.modules
if in_vscode:
    print("✅ VS Code 환경에서 실행 중입니다.")
    print("📌 참고: VS Code에서 위젯이 잘 작동하지 않으면 다음 확장을 설치해보세요:")
    print("   - Jupyter")
    print("   - Jupyter Notebook Renderers")

# 설치 확인
missing_packages = []
for package in required_packages:
    try:
        package_name = package.split('>=')[0].split('[')[0]
        importlib.import_module(package_name)
    except ImportError:
        missing_packages.append(package)

if missing_packages:
    print(f"⚠️ 다음 패키지를 가져올 수 없습니다. 수동으로 설치가 필요할 수 있습니다: {', '.join(missing_packages)}")
else:
    print("✓ 모든 패키지를 성공적으로 가져왔습니다. 작업을 계속 진행하세요.")

ValueError: 'module://ipympl.backend_nbagg' is not a valid value for backend; supported values are ['gtk3agg', 'gtk3cairo', 'gtk4agg', 'gtk4cairo', 'macosx', 'nbagg', 'notebook', 'qtagg', 'qtcairo', 'qt5agg', 'qt5cairo', 'tkagg', 'tkcairo', 'webagg', 'wx', 'wxagg', 'wxcairo', 'agg', 'cairo', 'pdf', 'pgf', 'ps', 'svg', 'template', 'inline', 'module://matplotlib_inline.backend_inline']

## 1. 필요한 라이브러리 및 모듈 가져오기

In [22]:
import sys
import time
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, clear_output
import ipywidgets as widgets

# 한글 폰트 설정
import matplotlib.font_manager as fm
import platform
import os

# 운영체제별 기본 한글 폰트 설정
if platform.system() == 'Windows':
    # 윈도우의 경우 맑은고딕 폰트 사용
    font_path = os.path.join(os.environ['SYSTEMROOT'], 'Fonts', 'malgun.ttf')
    if os.path.exists(font_path):
        # 폰트 등록
        font_prop = fm.FontProperties(fname=font_path)
        plt.rcParams['font.family'] = 'Malgun Gothic'
        print("윈도우 한글 폰트 설정 완료: 맑은 고딕")
    else:
        # 맑은 고딕이 없는 경우 대체 방법
        plt.rcParams['font.family'] = 'NanumGothic, Arial, sans-serif'
        print("기본 폰트 사용, 한글이 깨질 수 있습니다.")
elif platform.system() == 'Darwin':  # macOS
    plt.rcParams['font.family'] = 'AppleGothic'
    print("macOS 한글 폰트 설정 완료: Apple Gothic")
else:  # Linux
    # 나눔고딕이 설치된 경우
    font_path = '/usr/share/fonts/truetype/nanum/NanumGothic.ttf'
    if os.path.exists(font_path):
        font_prop = fm.FontProperties(fname=font_path)
        plt.rcParams['font.family'] = 'NanumGothic'
        print("Linux 한글 폰트 설정 완료: 나눔고딕")
    else:
        # 대체 방법
        print("한글 폰트를 찾을 수 없습니다. 한글이 깨질 수 있습니다.")

# 그래프에 마이너스 폰트 깨짐 방지
plt.rcParams['axes.unicode_minus'] = False

print("matplotlib 폰트 설정 완료")

# 프로젝트 모듈 가져오기 (경로 확인 필요)
sys.path.append('..')
try:
    from src.capture.screen_capture import ScreenCapture
    from src.detection.custom_detector import CustomDetector
    print("모듈 가져오기 성공!")
except ImportError as e:
    print(f"모듈 가져오기 실패: {e}")
    print("경로를 확인해주세요.")

윈도우 한글 폰트 설정 완료: 맑은 고딕
matplotlib 폰트 설정 완료
모듈 가져오기 성공!


## 2. 화면 캡처 기능 테스트

In [ ]:
# 화면 캡처 객체 생성
screen_capture = ScreenCapture()

# 화면 캡처 테스트
def test_screen_capture():
    print("화면 캡처 중...")
    start_time = time.time()
    
    # 화면 캡처
    frame = screen_capture.capture()
    
    end_time = time.time()
    print(f"캡처 완료! 소요 시간: {end_time - start_time:.4f}초")
    
    if frame is not None:
        print(f"캡처된 이미지 크기: {frame.shape}")
        
        # 이미지 표시
        plt.figure(figsize=(10, 6))
        plt.imshow(frame[:, :, ::-1])  # BGR -> RGB 변환
        plt.title('캡처된 화면')
        plt.axis('off')
        plt.show()
    else:
        print("화면 캡처에 실패했습니다.")
        
    return frame

# 화면 캡처 테스트 실행
captured_frame = test_screen_capture()

## 3. HSV 변환 및 필터링 테스트

In [ ]:
# CustomDetector 객체 생성
detector = CustomDetector()

# HSV 변환 및 필터링 테스트
def test_hsv_filtering(frame):
    print("HSV 변환 및 필터링 중...")
    
    if frame is None:
        print("유효한 프레임이 없습니다. 먼저 화면 캡처를 수행하세요.")
        return None
    
    # 현재 HSV 범위 출력
    print(f"현재 HSV 범위:")
    print(f"H: {detector.lower_color[0]} ~ {detector.upper_color[0]}")
    print(f"S: {detector.lower_color[1]} ~ {detector.upper_color[1]}")
    print(f"V: {detector.lower_color[2]} ~ {detector.upper_color[2]}")
    
    # 객체 검출
    result = detector.detect(frame)
    
    # 결과 표시
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    
    # 원본 이미지
    axes[0].imshow(frame[:, :, ::-1])  # BGR -> RGB
    axes[0].set_title('원본 이미지')
    axes[0].axis('off')
    
    # 마스크 이미지
    axes[1].imshow(result['mask'], cmap='gray')
    axes[1].set_title('HSV 필터링 마스크')
    axes[1].axis('off')
    
    # 바운딩 박스 이미지
    axes[2].imshow(result['bbox_frame'][:, :, ::-1])  # BGR -> RGB
    axes[2].set_title('바운딩 박스')
    axes[2].axis('off')
    
    plt.tight_layout()
    plt.show()
    
    return result

# 캡처된 프레임이 있다면 HSV 필터링 테스트 진행
if captured_frame is not None:
    result = test_hsv_filtering(captured_frame)
else:
    print("먼저 화면 캡처를 수행하세요.")

## 4. HSV 색상 범위 조정 위젯

In [ ]:
# HSV 조절 슬라이더 생성
h_min = widgets.IntSlider(value=detector.lower_color[0], min=0, max=255, description='H min:')
h_max = widgets.IntSlider(value=detector.upper_color[0], min=0, max=255, description='H max:')
s_min = widgets.IntSlider(value=detector.lower_color[1], min=0, max=255, description='S min:')
s_max = widgets.IntSlider(value=detector.upper_color[1], min=0, max=255, description='S max:')
v_min = widgets.IntSlider(value=detector.lower_color[2], min=0, max=255, description='V min:')
v_max = widgets.IntSlider(value=detector.upper_color[2], min=0, max=255, description='V max:')

# 결과 출력 영역
output = widgets.Output()

# 필터링을 실행하는 함수
def update_hsv_filter(frame, h_min, h_max, s_min, s_max, v_min, v_max):
    if frame is None:
        with output:
            print("유효한 프레임이 없습니다.")
        return
    
    # HSV 범위 업데이트
    detector.lower_color = np.array([h_min, s_min, v_min], dtype=np.uint8)
    detector.upper_color = np.array([h_max, s_max, v_max], dtype=np.uint8)
    
    # 객체 검출
    result = detector.detect(frame)
    
    with output:
        clear_output(wait=True)
        
        # 현재 HSV 범위 출력
        print(f"현재 HSV 범위:")
        print(f"H: {detector.lower_color[0]} ~ {detector.upper_color[0]}")
        print(f"S: {detector.lower_color[1]} ~ {detector.upper_color[1]}")
        print(f"V: {detector.lower_color[2]} ~ {detector.upper_color[2]}")
        
        # 결과 표시
        fig, axes = plt.subplots(1, 3, figsize=(15, 5))
        
        # 원본 이미지
        axes[0].imshow(frame[:, :, ::-1])
        axes[0].set_title('원본 이미지')
        axes[0].axis('off')
        
        # 마스크 이미지
        axes[1].imshow(result['mask'], cmap='gray')
        axes[1].set_title('HSV 필터링 마스크')
        axes[1].axis('off')
        
        # 바운딩 박스 이미지
        axes[2].imshow(result['bbox_frame'][:, :, ::-1])
        axes[2].set_title('바운딩 박스')
        axes[2].axis('off')
        
        plt.tight_layout()
        plt.show()

# 상호작용 위젯 생성
def create_interactive_widgets(frame):
    # 상호 작용 함수 생성
    interactive_update = widgets.interactive(
        lambda h_min_val, h_max_val, s_min_val, s_max_val, v_min_val, v_max_val: 
            update_hsv_filter(frame, h_min_val, h_max_val, s_min_val, s_max_val, v_min_val, v_max_val),
        h_min_val=h_min,
        h_max_val=h_max,
        s_min_val=s_min,
        s_max_val=s_max,
        v_min_val=v_min,
        v_max_val=v_max
    )
    
    # 위젯과 출력 표시
    display(widgets.VBox([widgets.HTML("<h3>HSV 색상 범위 조절</h3>"), 
                         widgets.HBox([widgets.VBox([h_min, h_max]), 
                                      widgets.VBox([s_min, s_max]), 
                                      widgets.VBox([v_min, v_max])])]))
    display(output)
    
    # 초기 업데이트 실행
    update_hsv_filter(frame, h_min.value, h_max.value, s_min.value, s_max.value, v_min.value, v_max.value)

# 캡처된 프레임이 있다면 위젯 생성
if captured_frame is not None:
    create_interactive_widgets(captured_frame)
else:
    print("먼저 화면 캡처를 수행하세요.")

Output()

## 5. 실시간 모니터링 테스트

In [ ]:
# 모니터링 시간 설정 (초)
monitoring_time = 10

# 실시간 모니터링 함수
def realtime_monitor(seconds=10, fps=5):
    print(f"실시간 모니터링 시작 ({seconds}초 동안 실행)...")
    print("중지하려면 Interrupt the kernel을 사용하세요.")
    
    # 모니터링 출력 영역
    monitor_output = widgets.Output()
    display(monitor_output)
    
    # 현재 HSV 설정값 확인
    print(f"현재 HSV 범위:")
    print(f"H: {detector.lower_color[0]} ~ {detector.upper_color[0]}")
    print(f"S: {detector.lower_color[1]} ~ {detector.upper_color[1]}")
    print(f"V: {detector.lower_color[2]} ~ {detector.upper_color[2]}")
    
    start_time = time.time()
    frame_count = 0
    
    try:
        while (time.time() - start_time) < seconds:
            # 화면 캡처
            frame = screen_capture.capture()
            
            if frame is None:
                time.sleep(0.1)
                continue
                
            # 객체 검출
            result = detector.detect(frame)
            
            with monitor_output:
                clear_output(wait=True)
                
                # 결과 표시
                fig, axes = plt.subplots(1, 3, figsize=(15, 5))
                
                # 원본 이미지
                axes[0].imshow(frame[:, :, ::-1])
                axes[0].set_title('실시간 캡처')
                axes[0].axis('off')
                
                # 마스크 이미지
                axes[1].imshow(result['mask'], cmap='gray')
                axes[1].set_title('HSV 필터링 마스크')
                axes[1].axis('off')
                
                # 바운딩 박스 이미지
                axes[2].imshow(result['bbox_frame'][:, :, ::-1])
                axes[2].set_title('바운딩 박스')
                axes[2].axis('off')
                
                plt.tight_layout()
                plt.show()
                
                elapsed = time.time() - start_time
                print(f"경과 시간: {elapsed:.1f}초 / {seconds}초")
                print(f"FPS: {frame_count/elapsed:.1f}")
                
            frame_count += 1
            time.sleep(1/fps)  # 프레임 레이트 조절
            
    except KeyboardInterrupt:
        print("사용자가 모니터링을 중지했습니다.")
    finally:
        elapsed_time = time.time() - start_time
        print(f"모니터링 완료: {frame_count}프레임, {elapsed_time:.2f}초, 평균 {frame_count/elapsed_time:.2f} FPS")

# 실시간 모니터링 시작 버튼
monitoring_button = widgets.Button(
    description='실시간 모니터링 시작',
    button_style='primary',
    tooltip='실시간 모니터링을 시작합니다.'
)

# 모니터링 시간 슬라이더
time_slider = widgets.IntSlider(
    value=10,
    min=5,
    max=30,
    step=1,
    description='모니터링 시간(초):',
    style={'description_width': 'initial'}
)

# fps 슬라이더
fps_slider = widgets.IntSlider(
    value=5,
    min=1,
    max=30,
    step=1,
    description='FPS:',
)

# 버튼 클릭 이벤트 핸들러
def on_button_clicked(b):
    realtime_monitor(time_slider.value, fps_slider.value)

monitoring_button.on_click(on_button_clicked)

# 위젯 표시
display(widgets.VBox([widgets.HTML("<h3>실시간 모니터링 설정</h3>"), 
                       widgets.HBox([time_slider, fps_slider]), 
                       monitoring_button]))

## 6. HSV 색상 범위 저장 및 불러오기

In [ ]:
import json
import os

def save_hsv_settings(name="default"):
    """HSV 설정을 JSON 파일로 저장"""
    settings = {
        "name": name,
        "hsv_lower": detector.lower_color.tolist(),
        "hsv_upper": detector.upper_color.tolist(),
    }
    
    filename = f"hsv_settings_{name}.json"
    with open(filename, 'w') as f:
        json.dump(settings, f, indent=2)
    
    print(f"HSV 설정을 '{filename}'에 저장했습니다.")

def load_hsv_settings(name="default"):
    """HSV 설정을 JSON 파일에서 불러오기"""
    filename = f"hsv_settings_{name}.json"
    
    if not os.path.exists(filename):
        print(f"'{filename}' 파일이 존재하지 않습니다.")
        return False
    
    try:
        with open(filename, 'r') as f:
            settings = json.load(f)
            
        detector.lower_color = np.array(settings["hsv_lower"], dtype=np.uint8)
        detector.upper_color = np.array(settings["hsv_upper"], dtype=np.uint8)
        
        print(f"'{filename}'에서 HSV 설정을 불러왔습니다.")
        print(f"현재 HSV 범위:")
        print(f"H: {detector.lower_color[0]} ~ {detector.upper_color[0]}")
        print(f"S: {detector.lower_color[1]} ~ {detector.upper_color[1]}")
        print(f"V: {detector.lower_color[2]} ~ {detector.upper_color[2]}")
        return True
    except Exception as e:
        print(f"설정을 불러오는 중 오류가 발생했습니다: {e}")
        return False

# HSV 설정 저장 위젯
save_name = widgets.Text(
    value='default',
    description='저장 이름:',
    style={'description_width': 'initial'}
)

save_button = widgets.Button(
    description='HSV 설정 저장',
    button_style='success',
    tooltip='현재 HSV 설정을 저장합니다.'
)

def on_save_button_clicked(b):
    save_hsv_settings(save_name.value)

save_button.on_click(on_save_button_clicked)

# HSV 설정 불러오기 위젯
load_name = widgets.Text(
    value='default',
    description='불러올 이름:',
    style={'description_width': 'initial'}
)

load_button = widgets.Button(
    description='HSV 설정 불러오기',
    button_style='info',
    tooltip='저장된 HSV 설정을 불러옵니다.'
)

def on_load_button_clicked(b):
    load_hsv_settings(load_name.value)
    
    # 슬라이더 업데이트 (있는 경우)
    try:
        h_min.value = detector.lower_color[0]
        h_max.value = detector.upper_color[0]
        s_min.value = detector.lower_color[1]
        s_max.value = detector.upper_color[1]
        v_min.value = detector.lower_color[2]
        v_max.value = detector.upper_color[2]
    except NameError:
        pass

load_button.on_click(on_load_button_clicked)

# 위젯 표시
display(widgets.HTML("<h3>HSV 설정 저장 및 불러오기</h3>"))
display(widgets.HBox([save_name, save_button]))
display(widgets.HBox([load_name, load_button]))

## 7. 정적 이미지 테스트 (사용자 캡처)

현재 화면을 캡처하고 HSV 필터링을 테스트합니다.

In [ ]:
# 현재 화면 캡처 버튼
capture_button = widgets.Button(
    description='화면 캡처',
    button_style='warning',
    tooltip='현재 화면을 캡처합니다.'
)

# 캡처 결과를 저장할 변수
static_frame = None

# 캡처 버튼 클릭 이벤트 핸들러
def on_capture_button_clicked(b):
    global static_frame
    print("화면 캡처 중...")
    static_frame = screen_capture.capture()
    
    if static_frame is not None:
        print(f"캡처 성공! 이미지 크기: {static_frame.shape}")
        
        # 이미지 표시
        plt.figure(figsize=(10, 6))
        plt.imshow(static_frame[:, :, ::-1])  # BGR -> RGB 변환
        plt.title('캡처된 정적 이미지')
        plt.axis('off')
        plt.show()
        
        # HSV 필터링 테스트 자동 실행
        test_hsv_filtering(static_frame)
    else:
        print("화면 캡처에 실패했습니다.")

capture_button.on_click(on_capture_button_clicked)

# 위젯 표시
display(widgets.HTML("<h3>정적 이미지 테스트</h3>"))
display(capture_button)